In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from bokeh.plotting import output_file
hv.extension('bokeh')

### Intro 

Script to reproduce the "[Warming Stripes](https://www.climate-lab-book.ac.uk/2018/warming-stripes/)" from [Ed Hawkins](http://www.met.reading.ac.uk/~ed/home/index.php).

### Read the data 

In [ ]:
# GISTEMP data from https://data.giss.nasa.gov/gistemp/tabledata_v3/GLB.Ts+dSST.csv
# If you have the data, use:
# df = pd.read_csv('GLB.Ts+dSST.csv', header=1, skipfooter=1, engine='python')

# This downloads it:
import requests, io
url = 'https://data.giss.nasa.gov/gistemp/tabledata_v3/GLB.Ts+dSST.csv'
response = requests.get(url)
df = pd.read_csv(io.StringIO(response.text), header=1, skipfooter=1, engine='python')

In [ ]:
# Annual values only
dfa = df[['Year', 'J-D']].iloc[:-1].copy()
dfa.columns = ['Year', 'Anomaly']
# This is to trick holoviews into making an image out of the dataframe
dfa['index'] = 1
# We want to display the rank as well
dfa['Rank'] = len(dfa) - np.argsort(dfa['Anomaly'])

In [ ]:
# Monthly values
dfm = df[df.columns[:13]].copy()
dfm = pd.melt(dfm, id_vars='Year', var_name='Month', value_name='Anomaly')

### Matplotlib plots 

In [ ]:
dfa['Anomaly'].plot();

In [ ]:
plt.pcolormesh(dfa['Anomaly'].values.reshape((1, len(dfa))), cmap='RdBu_r')
plt.colorbar();

### Bokeh plot: annual stripes

In [ ]:
# Display only these three columns as hover
hover = HoverTool(
        tooltips=[
            ("Year", "@Year"),
            ("Anomaly", "@Anomaly"),
            ("Rank", "@Rank"),
        ]
    )

In [ ]:
# simple trick to workaround https://github.com/ioam/holoviews/issues/2730
def set_active_drag(plot, element):
    plot.state.toolbar.active_drag = None

In [ ]:
# Display the plot
heatmap = hv.HeatMap(dfa, kdims=['Year', 'index'], 
                     label='NASA GISTEMP 1880-2017: annual deviation from 1951-1980 means')
heatmap = heatmap.options(tools=[hover], cmap='RdBu_r', width=700, height=300,
                xaxis=None, labelled=[], yaxis=None, toolbar=None,
                finalize_hooks=[set_active_drag]) 
heatmap

In [ ]:
# Save as HTML
renderer = hv.renderer('bokeh')
renderer.save(heatmap, 'annual-stripes-700x300')

In [ ]:
# Save larger plot
heatmap = hv.HeatMap(dfa, kdims=['Year', 'index'], 
                     label='NASA GISTEMP 1880-2017: annual deviation from 1951-1980 means')
heatmap = heatmap.options(tools=[hover], cmap='RdBu_r', width=1200, height=600,
                xaxis=None, labelled=[], yaxis=None, toolbar=None,
                finalize_hooks=[set_active_drag]) 
renderer = hv.renderer('bokeh')
renderer.save(heatmap, 'annual-stripes-1200x600')

### Bokeh plot: monthly stripes

In [ ]:
# Display the plot
heatmap = hv.HeatMap(dfm, kdims=['Year', 'Month'], 
                     label='NASA GISTEMP 1880-2017: monthly deviation from 1951-1980 means')
heatmap = heatmap.options(tools=['hover'], cmap='RdBu_r', width=700, height=300,
                xaxis=None, labelled=[], yaxis=None, toolbar=None,
                finalize_hooks=[set_active_drag]) 
heatmap

In [ ]:
# Save as HTML
renderer = hv.renderer('bokeh')
renderer.save(heatmap, 'monthly-stripes-700x300')

In [ ]:
# Save larger plot
heatmap = hv.HeatMap(dfm, kdims=['Year', 'Month'], 
                     label='NASA GISTEMP 1880-2017: monthly deviation from 1951-1980 means')
heatmap = heatmap.options(tools=['hover'], cmap='RdBu_r', width=1200, height=600,
                xaxis=None, labelled=[], yaxis=None, toolbar=None,
                finalize_hooks=[set_active_drag]) 
renderer = hv.renderer('bokeh')
renderer.save(heatmap, 'monthly-stripes-1200x600')